In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stomach_Cancer"
cohort = "GSE172197"

# Input paths
in_trait_dir = "../../input/GEO/Stomach_Cancer"
in_cohort_dir = "../../input/GEO/Stomach_Cancer/GSE172197"

# Output paths
out_data_file = "../../output/preprocess/Stomach_Cancer/GSE172197.csv"
out_gene_data_file = "../../output/preprocess/Stomach_Cancer/gene_data/GSE172197.csv"
out_clinical_data_file = "../../output/preprocess/Stomach_Cancer/clinical_data/GSE172197.csv"
json_path = "../../output/preprocess/Stomach_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE172197_family.soft.gz', 'GSE172197_series_matrix.txt.gz']
Identified SOFT files: ['GSE172197_family.soft.gz']
Identified matrix files: ['GSE172197_series_matrix.txt.gz']

Background Information:
!Series_title	"mRNA expression profiles of newly established 49 gastric cancer cell lines."
!Series_summary	"Establishment and molecular characterization of 49 peritoneally-metastatic gastric cancer cell lines from 18 patients’ ascites."
!Series_summary	"We performed comprehensive transcriptome analyses using microarrays of our established gastric cancer cell lines."
!Series_overall_design	"49 cancer cell lines"

Sample Characteristics Dictionary:
{0: ['cell line: NSC-10C', 'cell line: NSC-10X1A', 'cell line: NSC-10X1aA', 'cell line: NSC-10X1aF', 'cell line: NSC-10X1aX1', 'cell line: NSC-10X1aX1a', 'cell line: NSC-10X1F', 'cell line: NSC-11C', 'cell line: NSC-11X1', 'cell line: NSC-11X1a', 'cell line: NSC-15CA', 'cell line: NSC-15CF', 'cell line: NSC-16C', 'c

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze dataset based on background information and sample characteristics

# 1. Gene Expression Data Availability
# From the background information, this dataset contains "mRNA expression profiles" of gastric cancer cell lines
# and mentions "comprehensive transcriptome analyses using microarrays", indicating gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics, there's no explicit disease/control status, age, or gender info
# The cell lines are derived from gastric cancer, but they're all cancer cell lines without healthy controls
trait_row = None  # No trait data (cancer vs. control) available
age_row = None    # No age data available
gender_row = None  # No gender data available

# 2.2 Data Type Conversion Functions (even though we don't have the data, we define these for completeness)
def convert_trait(value):
    # Since there's no trait data, this function won't be used
    if value is None:
        return None
    
    value = value.split(":", 1)[1].strip() if ":" in value else value.strip()
    
    if "cancer" in value.lower():
        return 1
    elif "normal" in value.lower() or "control" in value.lower() or "healthy" in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    # Since there's no age data, this function won't be used
    if value is None:
        return None
    
    value = value.split(":", 1)[1].strip() if ":" in value else value.strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    # Since there's no gender data, this function won't be used
    if value is None:
        return None
    
    value = value.split(":", 1)[1].strip() if ":" in value else value.strip()
    
    if value.lower() in ["female", "f"]:
        return 0
    elif value.lower() in ["male", "m"]:
        return 1
    else:
        return None

# 3. Save Metadata
# The dataset has gene expression data but no trait data (no control samples)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')

Gene expression data shape: (54675, 49)


### Step 4: Gene Identifier Review

In [5]:
# Review gene identifiers

# The identifiers in this dataset (like '1007_s_at', '1053_at', etc.) are Affymetrix probe IDs
# from a microarray platform, not human gene symbols.
# These are probe set IDs that need to be mapped to official gene symbols.

# Microarray platforms like Affymetrix use these probe IDs which need to be converted
# to standard gene symbols before analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# Looking at the annotation data from the previous step, we can see:
# 'ID' column contains probe IDs like '1007_s_at' which match the gene expression data indices
# 'Gene Symbol' column contains the gene symbols we need for mapping

# 1. Identify the columns for mapping
probe_id_column = 'ID'
gene_symbol_column = 'Gene Symbol'

print(f"Will map from {probe_id_column} to {gene_symbol_column}")

# 2. Get a gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, probe_id_column, gene_symbol_column)
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few genes and their expression values:")
print(gene_data.head())

# Check the number of unique genes after mapping
print(f"Number of unique genes after mapping: {len(gene_data.index.unique())}")


Will map from ID to Gene Symbol
Gene mapping dataframe shape: (45782, 2)
First few rows of mapping dataframe:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A


Gene expression data shape after mapping: (21278, 49)
First few genes and their expression values:
           GSM5243830  GSM5243831   GSM5243832   GSM5243833   GSM5243834  \
Gene                                                                       
A1BG         3.573000   68.113460    90.104184    30.680121    93.146829   
A1BG-AS1    40.920908    9.646156    32.501140    99.996111     5.923232   
A1CF      7221.275999  854.425711  3318.552915  1904.586094  2881.262597   
A2M        139.724070  200.880079   188.067771   241.584986   145.263539   
A2M-AS1     74.847679  152.428161   216.075561   211.613157   191.989181   

           GSM5243835   GSM5243836   GSM5243844   GSM5243845   GSM5243846  \
Gene                                                                        
A1BG        16.142043    38.469045    18.348572   241.560810   147.073023   
A1BG-AS1     9.977724    16.494795    16.934687    13.573422    17.666532   
A1CF      2768.575985  2619.561153  4297.496644  4920.868032

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Create a minimal clinical DataFrame with sample IDs
# We don't have trait data as determined in Step 2, but we still need to follow the process
sample_ids = normalized_gene_data.columns
clinical_features = pd.DataFrame(index=sample_ids)

# Add placeholder for trait column (all labeled as 1 since all samples are gastric cancer)
clinical_features[trait] = 1  # All samples are gastric cancer cell lines

# Save the clinical data even though it's minimal
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link the clinical and genetic data (even though clinical data is minimal)
linked_data = geo_link_clinical_genetic_data(clinical_features.T, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values
# Since all our samples are cancer cell lines with the same trait value (1),
# and we don't have age/gender data, we can just check for missing values in gene data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait is biased
# Since all samples are cancer cell lines (no controls), the trait is completely biased
is_trait_biased = True
print("All samples are gastric cancer cell lines without controls, marking trait as biased.")

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,  # We have trait data (all cancer), though it's biased
    is_biased=is_trait_biased, 
    df=linked_data,
    note="Dataset contains gene expression data from gastric cancer cell lines but lacks control samples for comparison."
)

# 6. We've determined the data is not usable for association studies due to biased trait
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # This block likely won't execute but included for completeness
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset contains only gastric cancer samples without controls, making it unsuitable for association studies.")

Normalized gene data shape: (19845, 49)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Normalized gene data saved to ../../output/preprocess/Stomach_Cancer/gene_data/GSE172197.csv
Clinical data saved to ../../output/preprocess/Stomach_Cancer/clinical_data/GSE172197.csv
Linked data shape: (49, 19846)


Linked data shape after handling missing values: (49, 19846)
All samples are gastric cancer cell lines without controls, marking trait as biased.
Data quality check result: Not usable
Data quality check failed. The dataset contains only gastric cancer samples without controls, making it unsuitable for association studies.
